In [7]:
# Basic DS stuff
import numpy as np
import pandas as pd
import json

# Web scraping
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import asyncio

# For labeling records, tracking files, and formatting
from datetime import datetime
import time

# For Rick
import pickle

In [2]:
# NOTE: Use this cell to reset the scraper, making it forget all records.

# game_mapped_users = set([0])
# with open('../data/raw/game_mapped_users.pkl', 'rb+') as file :
#     pickle.dump(game_mapped_users, file)

# with open('../data/raw/recently_played.pkl', 'wb+') as file :
#     pickle.dump(set([0]), file)

In [3]:
# Load data

with open('../data/raw/all_users', 'rb+') as file :
    all_users = set(pickle.load(file))

with open('../data/raw/game_mapped_users.pkl', 'rb+') as file :
    game_mapped_users = pickle.load(file)
    
# with open('../data/raw/skipped_game_map_users', 'rb+') as file :
#     skipped_users = pickle.load(file)

In [4]:
# Create vars

holding_set = set()

unmapped_users = list(all_users-game_mapped_users)

key = '83A112A706DF72CB0920A20F28F04252'

interval = 10

base_url = 'http://api.steampowered.com/IPlayerService/GetRecentlyPlayedGames/v0001/?key={}&steamid={}&format=json'

-------------
### This is the API version

In [5]:
async def get_games(user, url) :
    return (user, urlopen(url))

In [6]:
to_scrape = len(unmapped_users)

for i in range(0, to_scrape, interval) :
    try :
        iter_max = min(i+interval, to_scrape)
        users = [unmapped_users[i] for i in range(i, iter_max)]
        tasks = [get_games(user, base_url.format(key, user)) for user in users]
        responses = await asyncio.gather(*tasks)
        for response in responses :
            user = response[0]
            jason = json.load(response[1])
            games_list = jason['response']
            if len(games_list) > 0 :
                if games_list['total_count'] > 0 :
                    for game in games_list['games'] :
                        appid = game['appid']
                        playtime_2weeks = game['playtime_2weeks']
                        playtime_forever = game['playtime_forever']
                        holding_set.add((user, appid, playtime_2weeks, playtime_forever))
            game_mapped_users.add(user)
    except Exception as e:
        print(e)
        print(datetime.now())
        print('Trying again in 1 minute...')
        time.sleep(60)

    if i % 200 == 0 :
        with open('../data/raw/recently_played.pkl', 'rb+') as file :
            total_set = pickle.load(file)
        total_set.update(holding_set)
        with open('../data/raw/recently_played.pkl', 'wb+') as file :
            pickle.dump(total_set, file)
        with open('../data/raw/game_mapped_users.pkl', 'wb+') as file :
            pickle.dump(game_mapped_users, file)
        print(f'Added: {len(holding_set)}')
        print(f"Saved so far: {len(total_set)}")
        holding_set = set()
        print(datetime.now())
        print('')


Added: 0
Saved so far: 1148726
2024-03-24 18:54:29.028232

Added: 197
Saved so far: 1148923
2024-03-24 18:55:08.435344

Added: 211
Saved so far: 1149134
2024-03-24 18:56:19.657058

Added: 186
Saved so far: 1149320
2024-03-24 18:57:01.625573

Added: 167
Saved so far: 1149487
2024-03-24 18:57:41.231345

Added: 139
Saved so far: 1149626
2024-03-24 18:58:19.031499

Added: 166
Saved so far: 1149792
2024-03-24 18:58:53.071189

Added: 129
Saved so far: 1149921
2024-03-24 18:59:24.273589

Added: 172
Saved so far: 1150093
2024-03-24 18:59:59.647066

Added: 152
Saved so far: 1150245
2024-03-24 19:00:40.248213

Added: 137
Saved so far: 1150382
2024-03-24 19:01:15.393320

Added: 122
Saved so far: 1150504
2024-03-24 19:01:52.851559

Added: 176
Saved so far: 1150680
2024-03-24 19:02:25.001402

Added: 141
Saved so far: 1150821
2024-03-24 19:03:31.459991

Added: 157
Saved so far: 1150978
2024-03-24 19:04:10.673929

HTTP Error 502: Bad Gateway
2024-03-24 19:04:28.091991
Trying again in 1 minute...
Adde

CancelledError: 

In [ ]:
# This works but is deprecated. Saved as a potential reference.

# counter = 0
# game_counts = set()

# for i in range(100) :

#     user = all_users[i]

#     url = f'http://api.steampowered.com/IPlayerService/GetRecentlyPlayedGames/v0001/?key={key}&steamid={user}&format=json'
#     with urlopen(url) as response :
#         jason = json.load(response)
#     games_list = jason['response']
#     if len(games_list) > 0 :
#         if games_list['total_count'] > 0 :
#             print(jason)
#             print('\n')
#             counter +=1
#             game_counts.add(len(games_list))

# print(f'{counter} users had recently played games. Of them:')
# print(f'Avg games played: {sum(game_counts)/len(game_counts)}')
# print(f'Most games played: {max(game_counts)}')